## Adding the dependencies

In [18]:
import numpy as np 
import tensorflow
import sklearn
import pickle


In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,LSTM,Dense
from sklearn.feature_extraction.text import CountVectorizer

#initialize all variables 
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

### Parse the dataset file

We will traverse the dataset file and extract all the input and target texts. For this, we will be using the first 10,000 rows of our dataset for the training and testing part. It can be changed as per requirements. To separate input and target texts from the row we will use ‘\t’ and to separate rows we will use ‘\n’.


In [20]:
#read dataset file
with open('eng-french.txt','r',encoding='utf-8') as f:
    rows=f.read().split('\n')
#read first 10,000 rows from dataset     
for row in rows[:10000]:
    #split input and target by '\t'=tab
    input_text,target_text = row.split('\t')
    #add '\t' at start and '\n' at end of text.
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text.lower())
    target_texts.append(target_text.lower())
    #split character from text and add in respective sets
    input_characters.update(list(input_text.lower()))
    target_characters.update(list(target_text.lower()))

We will use the same procedure and separate the text from rows and characters. Also, get the maximum length of encoder as well as decoder sequence.

In [21]:
#sort input and target characters 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

#get the total length of input and target characters
num_en_chars = len(input_characters)
num_dec_chars = len(target_characters)

#get the maximum length of input and target text.
max_input_length = max([len(i) for i in input_texts])
max_target_length = max([len(i) for i in target_texts])

print("number of encoder characters : ",num_en_chars)
print("number of decoder characters : ",num_dec_chars)

print("maximum input length : ",max_input_length)
print("maximum target length : ",max_target_length)

number of encoder characters :  47
number of decoder characters :  67
maximum input length :  16
maximum target length :  59


Models cannot work directly on the categorical data. For this, we require one hot encoding process. One-hot encoding deals with the data in binary format so we encode the categorical data in binary format.

One-hot means that we can only make an index of data 1 (true) if it is present in the vector or else 0 (false). So every data has its unique representation in vector format.

In [35]:
def bagofcharacters(input_texts,target_texts):
  
    # Initialize lists to store the data
    en_in_data = []
    dec_in_data = []
    dec_tr_data = []

    #padding variable with first character as 1 as rest all 0.
    pad_en=[1]+[0]*(len(input_characters)-1)
    pad_dec=[0]*(len(target_characters)) ; pad_dec[2]=1

    #countvectorizer for one hot encoding as we want to tokenize character so
    #analyzer is true and None the stopwords action.
    cv=CountVectorizer(binary=True,tokenizer=lambda txt:
    txt.split(),stop_words=None,analyzer='char')
  
    for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):
        #fit the input characters into the CountVectorizer function
        cv_inp= cv.fit(input_characters)
    
        #transform the input text from the help of CountVectorizer fit.
        #it character present than put 1 and 0 otherwise.
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
        cv_tar= cv.fit(target_characters)    
        dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())

        #decoder target will be one timestep ahead because it will not consider 
        #the first character i.e. '\t'.
        dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
          #add padding variable if the length of the input or target text is smaller
  #than their respective maximum input or target length. 
    if len(input_t) < max_input_length:
      for _ in range(max_input_length-len(input_t)):
        en_in_data[i].append(pad_en)
    if len(target_t) < max_target_length:
      for _ in range(max_target_length-len(target_t)):
        dec_in_data[i].append(pad_dec)
    if (len(target_t)-1) < max_target_length:
      for _ in range(max_target_length-len(target_t)+1):
        dec_tr_data[i].append(pad_dec)

#convert list to numpy array with data type float32
    en_in_data=np.array(en_in_data,dtype="float32")
    dec_in_data=np.array(dec_in_data,dtype="float32")
    dec_tr_data=np.array(dec_tr_data,dtype="float32")
    return en_in_data,dec_in_data,dec_tr_data

### Build the training model

In this language translation project, we will be using LSTM to train our machine learning model to translater language. LSTM (Long Short Term Memory) network: LSTM is a type of RNN (Recurrent Neural Network) that solves scenarios where RNN is failed.

Long-Term Dependency: In RNN, networks have the data of previous output in memory for a short period of time because of this they are unaware about the actual context of the sentence over a long period of time. This raised the issue of long-term dependency.

In [30]:
#create input object of total number of encoder characters
en_inputs = Input(shape=(None, num_en_chars))

#create LSTM with the hidden dimension of 256
#return state=True as we don't want output sequence.
encoder = LSTM(256, return_state=True)

#discard encoder output and store hidden and cell state.
en_outputs, state_h, state_c = encoder(en_inputs)
en_states = [state_h, state_c]

In [31]:
#create input object of total number of decoder characters
dec_inputs = Input(shape=(None, num_dec_chars))

#create LSTM with the hidden dimension of 256
#return state and return sequences as we want output sequence.
dec_lstm = LSTM(256, return_sequences=True, return_state=True)

#initialize the decoder model with the states on encoder.
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=en_states)

#Output layer with shape of total number of decoder characters 
dec_dense = Dense(num_dec_chars, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

### Train the model

To train the model we will fit ‘(encoder input and decoder input)’ which will turn into (‘decoder target data’) using ‘Adam’ optimizer with a validation split of 0.2 and provide an epoch of 200 in a batch size of 64. Also, we will store all required variables in a binary or bytes stream like object format file using the ‘pickle’ module.

In [ ]:
#create Model and store all variables 
model = Model([en_inputs, dec_inputs], dec_outputs)

pickle.dump({'input_characters':input_characters,'target_characters':target_characters, 'max_input_length':max_input_length, 'max_target_length':max_target_length, 'num_en_chars':num_en_chars, 'num_dec_chars':num_dec_chars}, open("training_data.pkl", "wb"))

#load the data and train the model
en_in_data,dec_in_data,dec_tr_data = bagofcharacters(input_texts,target_texts)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [en_in_data, dec_in_data],
    dec_tr_data,
    batch_size=64,
    epochs=200,
    validation_split=0.2,
)

# Save model
model.save("s2s")

In [36]:
#summary and model plot
model.summary()

plot_model(model, to_file='Model_plot.png', show_shapes=True, show_layer_names=True)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, None, 47)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, None, 67)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 256),     │    311,296 │ input_layer_3[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, None,     │    331,776 │ input_layer_4[0]… │
│                     │ 256), (None,      │            │ lstm_3[0][1],     │
│                     │ 256), (None,      │            │ lstm_3[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 67)  │     17,219 │ lstm_4[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 660,291 (2.52 MB)

 Trainable params: 660,291 (2.52 MB)

 Non-trainable params: 0 (0.00 B)

You must install pydot (`pip install pydot`) for `plot_model` to work.


 ### Create Gui for our prediction

In [37]:
from tkinter import *
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Input,LSTM,Dense

BG_GRAY="#ABB2B9"
BG_COLOR="#000"
TEXT_COLOR="#FFF"
FONT="Melvetica 14"
FONT_BOLD="Melvetica 13 bold"

cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char') 

class LangTRans:
    def __init__(self):
        #initialize tkinter window and load the file
        self.window=Tk()
        self.main_window()
        self.datafile()

In [38]:
def datafile(self):
    #get all datas from datafile and load the model.

    datafile = pickle.load(open("training_data.pkl","rb"))
    self.input_characters = datafile['input_characters']
    self.target_characters = datafile['target_characters']

    self.max_input_length = datafile['max_input_length']
    self.max_target_length = datafile['max_target_length']
    self.num_en_chars = datafile['num_en_chars']
    self.num_dec_chars = datafile['num_dec_chars']

    self.loadmodel()

Create the main window for language translation. Also create scrollbar, text widget for the GUI:

In [39]:
def main_window(self):
    #add title to window and configure it
    self.window.title("Language Translator")
    self.window.resizable(width=False,height=False)
    self.window.configure(width=520,height=520,bg=BG_COLOR)

    head_label=Label(self.window,bg=BG_COLOR, fg=TEXT_COLOR, text="Welcome to DataFlair",font=FONT_BOLD, pady=10)
    head_label.place(relwidth=1)

    line = Label(self.window,width=450,bg=BG_COLOR)
    line.place(relwidth=1,rely=0.07,relheight=0.012)

    #create text widget where input and output will be displayed
    self.text_widget=Text(self.window,width=20,height=2,bg="#fff",fg="#000", font=FONT,padx=5,pady=5)
    self.text_widget.place(relheight=0.745,relwidth=1,rely=0.08)
    self.text_widget.configure(cursor="arrow",state=DISABLED)

    #create scrollbar
    scrollbar=Scrollbar(self.text_widget)
    scrollbar.place(relheight=1,relx=0.974)
    scrollbar.configure(command=self.text_widget.yview)

    #create bottom label where text widget will placed
    bottom_label=Label(self.window,bg=BG_GRAY,height=80)
    bottom_label.place(relwidth=1,rely=0.825)

    #this is for user to put english text
    self.msg_entry=Entry(bottom_label,bg="#2C3E50", fg=TEXT_COLOR,font=FONT)
    self.msg_entry.place(relwidth=0.788,relheight=0.06,rely=0.008,relx=0.008)
    self.msg_entry.focus()
    self.msg_entry.bind("<Return>",self.on_enter)

    #send button which will call on_enter function to send the text
    send_button=Button(bottom_label,text="Send",font=FONT_BOLD, width=8,bg="#fff",command=lambda: self.on_enter(None))        
    send_button.place(relx=0.80,rely=0.008,relheight=0.06,relwidth=0.20)

### Inference (sampling) model and prediction

Load the saved model and construct encoder and decoder. We will get the inputs from the saved model and LSTM to get the hidden and cell state of the encoder which is required to create the encoder model.


In [40]:
def loadmodel(self):
    #Inference model
    #load the model
    model = models.load_model("s2s")

    #construct encoder model from the output of second layer
    #discard the encoder output and store only states.
    enc_outputs, state_h_enc, state_c_enc = model.layers[2].output 

    #add input object and state from the layer.
    self.en_model = Model(model.input[0], [state_h_enc, state_c_enc])

For the decoder, we will take the second input and create an input object for hidden as well for cell state of shape (256,) which is latent(hidden) dimension of layer. Also, we will run one step of the decoder with this initial state and a start of text character after that our output will be the next character of the text.

We will use reverse lookup to get characters from the index of the ‘input_text’ variable.

In [42]:
#create Input object for hidden and cell state for decoder
#shape of layer with hidden or latent dimension
dec_state_input_h = Input(shape=(256,), name="input_3")
dec_state_input_c = Input(shape=(256,), name="input_4")
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

#add input from the encoder output and initialize with states.
dec_lstm = model.layers[3]
dec_outputs, state_h_dec, state_c_dec = dec_lstm(
    model.input[1], initial_state=dec_states_inputs
)

dec_states = [state_h_dec, state_c_dec]
dec_dense = model.layers[4]
dec_outputs = dec_dense(dec_outputs)

In [45]:
def decode_sequence(self,input_seq):
    #create a dictionary with a key as index and value as characters.
    reverse_target_char_index = dict(enumerate(self.target_characters))
    #get the states from the user input sequence
    states_value = self.en_model.predict(input_seq)

    #fit target characters and 
    #initialize every first character to be 1 which is '\t'.
    #Generate empty target sequence of length 1.
    co=cv.fit(self.target_characters) 
    target_seq=np.array([co.transform(list("\t")).toarray().tolist()],dtype="float32")

    #if the iteration reaches the end of text than it will be stop the it
    stop_condition = False
    #append every predicted character in decoded sentence
    decoded_sentence = ""

    while not stop_condition:
        #get predicted output and discard hidden and cell state.
        output_chars, h, c = self.dec_model.predict([target_seq] + states_value)

        #get the index and from the dictionary get the character.
        char_index = np.argmax(output_chars[0, -1, :])
        text_char = reverse_target_char_index[char_index]
        decoded_sentence += text_char
        # Exit condition: either hit max length
    # or find a stop character.
    if text_char == "\n" or len(decoded_sentence) > self.max_target_length:
        stop_condition = True
    #update target sequence to the current character index.
    target_seq = np.zeros((1, 1, self.num_dec_chars))
    target_seq[0, 0, char_index] = 1.0
    states_value = [h, c]
#return the decoded sentence
    return decoded_sentence

For every index, put 1 to that index of our target array. So for the next iteration, our target sequence will be having a vector of the previous character. Iterate until our character is equal to the last character or max length of the target text.

Get the input (English) text from the user and pass it to a bag of characters for a one-hot encoding process. After that pass the encoded vector into ‘decode_sequence()’ for the decoded(french) text.

In [51]:
def bagofcharacters(self, input_t):
    cv = CountVectorizer(binary=True, tokenizer=lambda txt: txt.split(), stop_words=None, analyzer='char')
    en_in_data = []
    pad_en = [1] + [0] * (len(self.input_characters) - 1)

    cv_inp = cv.fit(self.input_characters)
    en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())

    if len(input_t) < self.max_input_length:
        for _ in range(self.max_input_length - len(input_t)):
            en_in_data[0].append(pad_en)

    return np.array(en_in_data, dtype="float32")

def decoded_output(self, msg, sender):
    self.text_widget.configure(state=NORMAL)
    en_in_data = self.bagofcharacters(msg.lower() + ".")
    self.text_widget.insert(END, str(sender) + " : " + self.decode_sequence(en_in_data) + "\n\n")
    self.text_widget.configure(state=DISABLED)
    self.text_widget.see(END)

def my_msg(self, msg, sender):
    if not msg:
        return
    self.msg_entry.delete(0, END)
    self.text_widget.configure(state=NORMAL)
    self.text_widget.insert(END, str(sender) + " : " + str(msg) + "\n")
    self.text_widget.configure(state=DISABLED)

# Run window
def run(self):
    self.window.mainloop()
